In [1]:
import pandas as pd
%matplotlib inline
%run Pipeline//upload_and_vizualize 
%run Pipeline//classify_and_evaluate 
%run Pipeline//aux
%run Pipeline//ULAB_ML_Pipeline
%run processing
import numpy as np 
import matplotlib.pyplot as plt
import pylab
import sys
import random
import sklearn as sk 
import json 
import re
from multiprocessing import Pool
from functools import partial
from time import time
from sklearn import svm, ensemble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split, KFold
from sklearn.preprocessing import *
from sklearn.feature_selection import RFE
from sklearn.grid_search import ParameterGrid
from sklearn.svm import LinearSVC
from sklearn.metrics import *
import csv
from errno import EEXIST
from os import makedirs,path


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
FACILITY = 'ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv'
VIOLATION = 'ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv'
INSPECTION = 'ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv'
FORMALACT = 'ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv'
INFORMALACT = 'ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv'
STACKTEST = 'ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv'
TITLEV = 'ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv'

#facilities = read_file('ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv')
#pd_violations = read_file('ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
#pd_inspection = read_file('ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
#pd_formalact = read_file('ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv')
#pd_informalact = read_file('ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv')
#pd_stacktest = read_file('ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv')
#pd_titlev = read_file('ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv')

In [3]:
## FROM PREPROCESSING ##

#interest_var = ['PGM_SYS_ID','ACTIVITY_ID','AGENCY_TYPE_DESC','STATE_CODE','AIR_LCON_CODE','COMP_DETERMINATION_UID','ENF_RESPONSE_POLICY_CODE','PROGRAM_CODES']
def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)

def convert_to_year(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.year)

def to_date_time(df, date_format, date_col):
    #add datetime column
    df[date_col] = df[date_col].apply(convert_to_datetime, date_format=date_format)
    df[date_col+'_year'] = df[date_col].apply(convert_to_year)

    return df

def filter_date(df, date_format, date_col, start=None, end=None):
    df = to_date_time(df, date_format, date_col)
    
    
    if start:
        timestart = dt.strptime(start,"%Y/%m/%d")
        #print(start)
        df = df[df[date_col] >= timestart ]
        #print(df.head())
    if end:
        timeend = dt.strptime(end,"%Y/%m/%d")
        #print(end)
        df = df[df[date_col] <= timeend ]
        #print(df.head())
    
    return df

def filter_col(df, fac_id, features, date_col):
    #filter needed
    df = df[[fac_id] + [date_col] + [date_col+'_year'] + features]
    return df

In [4]:
## CONFIG DATA ##
START_DATE= '2007/01/01'
END_DATE = '2016/12/31'
fac_id = 'PGM_SYS_ID'
df_dict ={'violation': {'interest_var': ['AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_RESOLVED_DATE'],
                         'date_col': 'HPV_DAYZERO_DATE',
                       'date_format':'%m-%d-%Y'},
          
           'inspection': {'interest_var': ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES'],
                         'date_col': 'ACTUAL_END_DATE',
                         'date_format':'%m-%d-%Y'},
          
           'stacktest': {'interest_var':['COMP_MONITOR_TYPE_CODE','POLLUTANT_CODES','AIR_STACK_TEST_STATUS_CODE'],
                        'date_col': 'ACTUAL_END_DATE',
                        'date_format':'%m/%d/%Y'},
          
           'titlev':{'interest_var':['COMP_MONITOR_TYPE_CODE','FACILITY_RPT_DEVIATION_FLAG'],
                        'date_col': 'ACTUAL_END_DATE',
                    'date_format':'%m/%d/%Y'},
          
           'formalact':{'interest_var':['ENF_TYPE_CODE','PENALTY_AMOUNT'],
                        'date_col': 'SETTLEMENT_ENTERED_DATE',
                       'date_format':'%m/%d/%Y'},
          
           'informalact':{'interest_var':['ENF_TYPE_CODE'],
                        'date_col': 'ACHIEVED_DATE',
                         'date_format':'%m/%d/%Y'}}

#general_df = pd.DataFrame()

In [5]:
for i, v in df_dict.items():
    print(v['date_col'])

HPV_DAYZERO_DATE
ACTUAL_END_DATE
ACTUAL_END_DATE
ACTUAL_END_DATE
SETTLEMENT_ENTERED_DATE
ACHIEVED_DATE


In [6]:
def general_read_file(df_dict, start_date, end_date):
    #df = pd.DataFrame()
    
    violation = []
    inspection = []
    stacktest = []
    titlev = []
    formalact = []
    informalact = []
    
    for table, var in df_dict.items():
        date_col = var['date_col']
        features = var['interest_var']
        DATE_FORMAT = var['date_format']
        
        if table == 'violation':
            violation = read_file(VIOLATION)
            violation = violation[violation['ENF_RESPONSE_POLICY_CODE'] != 'FRV']
            
            violation = filter_date(violation, DATE_FORMAT, date_col, start=start_date, end=end_date)
            violation = filter_col(violation, fac_id, features, date_col)
        
        elif table == 'inspection':
            inspection = read_file(INSPECTION)
            
            inspection = filter_date(inspection, DATE_FORMAT, date_col, start=start_date, end=end_date)
            inspection = filter_col(inspection, fac_id, features, date_col)
        
        elif table == 'titlev':
            titlev = read_file(TITLEV)
            
            titlev = filter_date(titlev, DATE_FORMAT, date_col, start=start_date, end=end_date)
            titlev = filter_col(titlev, fac_id, features, date_col)
        
        elif table == 'stacktest':
            stacktest = read_file(STACKTEST)
            
            stacktest = filter_date(stacktest, DATE_FORMAT, date_col, start=start_date, end=end_date)
            stacktest = filter_col(stacktest, fac_id, features, date_col)
        
        elif table == 'formalact':
            formalact = read_file(FORMALACT)
            
            formalact = filter_date(formalact, DATE_FORMAT, date_col, start=start_date, end=end_date)
            formalact = filter_col(formalact, fac_id, features, date_col)
        
        elif table == 'informalact':
            informalact = read_file(INFORMALACT)
            
            informalact = filter_date(informalact, DATE_FORMAT, date_col, start=start_date, end=end_date)
            informalact = filter_col(informalact, fac_id, features, date_col)
        
    return (violation, inspection, titlev, stacktest, formalact, informalact)

In [7]:
## READ INITIAL FILTERED FILE ##
violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)

In [8]:
def add_dummy(df, variable_list, sep_char = None, drop_one=False, drop_original=False):
    '''
    Input: 
        - df: pandas dataframe
        - variable_list: a list of variables to dummitize
        - drop_one: whether to drop first dummy
        - drop_original: whether to drop original categorical variable
    Output: dataframe with tht dummy variables added
    '''
    for variable in variable_list:
        if sep_char:
            df_dummy = df[variable].str.get_dummies(sep=sep_char)
            df_dummy.columns = [variable+ '_' +str(col) for col in df_dummy.columns]

        else:
            df_dummy = pd.get_dummies(df[variable], drop_first=drop_one, prefix = variable)
        
        df = pd.concat([df, df_dummy], axis=1)
        if drop_original:
            df = df.drop(variable, 1)
    return (df, df_dummy.columns)

In [145]:
violation.isnull().sum()

PGM_SYS_ID                  0
HPV_DAYZERO_DATE            0
HPV_DAYZERO_DATE_year       0
AGENCY_TYPE_DESC            0
STATE_CODE                  0
AIR_LCON_CODE               0
ENF_RESPONSE_POLICY_CODE    0
POLLUTANT_CODES             0
PROGRAM_CODES               0
HPV_RESOLVED_DATE           0
dtype: int64

In [9]:
viol = violation
viol

,PGM_SYS_ID,HPV_DAYZERO_DATE,HPV_DAYZERO_DATE_year,AGENCY_TYPE_DESC,AIR_LCON_CODE,ENF_RESPONSE_POLICY_CODE,POLLUTANT_CODES,PROGRAM_CODES,HPV_RESOLVED_DATE
5,CT0000000900508907,2010-03-17,2010,State,NaN,HPV,300000329,CAAFESOP,11-02-2010
6,CT0000000900100078,2007-08-08,2007,State,NaN,HPV,300000329,CAAFESOP,09-24-2009
7,CT0000000901501125,2013-04-26,2013,State,NaN,HPV,10193,CAATVP,06-25-2014
9,CT0000000900109058,2008-01-17,2008,State,NaN,HPV,300000005,CAAFESOP,01-06-2009
13,CT0000000901100054,2008-04-08,2008,State,NaN,HPV,10461 300000005,CAATVP,12-10-2008
16,CT0000000901100038,2009-10-01,2009,State,NaN,HPV,300000005,CAASIP,03-25-2011
18,CT0000000900900016,2010-04-26,2010,State,NaN,HPV,300000329,CAANSPS CAANSR CAASIP CAATVP,10-15-2012
24,MA0000002512000866,2010-03-01,2010,State,NaN,HPV,10461,CAASIP CAATVP,12-31-2011
29,CT0000000900909910,2010-07-27,2010,State,NaN,HPV,300000298,CAANSR,08-05-2011
30,CT0000000900909936,2010-05-18,2010,State,NaN,HPV,300000005,CAASIP CAATVP,02-01-2011


In [20]:
def process_violation(violation_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    outcome = ['ENF_RESPONSE_POLICY_CODE']
    cat_var = ['AGENCY_TYPE_DESC', 'AIR_LCON_CODE']
    dum_var = ['PROGRAM_CODES', 'POLLUTANT_CODES']
    
    df = violation_df[violation_df['HPV_DAYZERO_DATE_year'] >= start_year]
    df = df[df['HPV_DAYZERO_DATE_year'] <= end_year]
    
    ## Replace NaN with 'None' (string) --> Making 'None' it's own category
    nan = df.columns[df.isnull().any()].tolist()
    values = ['None']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize 
    df, colnames_out = add_dummy(df, outcome, drop_original = True)
    df, colnames_dum = add_dummy(df, dum_var, sep_char = ' ')
    
    for i, cat in enumerate(cat_var):
        #print(i)
        cross = pd.crosstab(df['PGM_SYS_ID'], df[cat])
        cross.columns = [cross.columns.name+ '_' +str(col) for col in cross.columns]
        cross.columns.name = None
        cross.reset_index(inplace=True)
        #print(cross)
        if i == 0:
            final_df = final_df.append(cross)
            print(final_df.columns)
        else:
            final_df = pd.merge(final_df, cross, how = 'left', on = 'PGM_SYS_ID')
    
    for dum in colnames_dum:
        cross = pd.crosstab(df['PGM_SYS_ID'], df[dum])
        
        #cross = cross.rename(columns={0 : "drop"})
        print(cross.columns)
        cross = cross.drop(0, axis = 1)
        cross.columns = [cross.columns.name for col in cross.columns]
        cross.columns.name = None
        cross.reset_index(inplace=True)

        final_df = pd.merge(final_df, cross, how = 'left', on = 'PGM_SYS_ID')
    
    for out in colnames_out:
        cross = pd.crosstab(df['PGM_SYS_ID'], df[dum])
        cross.columns = [cross.columns.name for col in cross.columns]
        cross.columns.name = None
        cross.reset_index(inplace=True)

        final_df = pd.merge(final_df, cross, how = 'left', on = 'PGM_SYS_ID')
    
    return final_df

    

In [21]:
process_violation(viol, '2010', '2016')

Index(['PGM_SYS_ID', 'AGENCY_TYPE_DESC_Local', 'AGENCY_TYPE_DESC_State',
       'AGENCY_TYPE_DESC_U.S. EPA'],
      dtype='object')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10193')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10214')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10243')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10305')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10358')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10361')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10366')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10373')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10461')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10486')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_10494')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_12289')
Int64Index([0, 1], dtype='int64', name='POLLUTANT_CODES_200000481')
Int64Index([0, 1], dtype='int64'

,PGM_SYS_ID,AGENCY_TYPE_DESC_Local,AGENCY_TYPE_DESC_State,AGENCY_TYPE_DESC_U.S. EPA,AIR_LCON_CODE_ACH,AIR_LCON_CODE_AVA,AIR_LCON_CODE_BAA,AIR_LCON_CODE_BUN,AIR_LCON_CODE_CCH,AIR_LCON_CODE_CHC,...,POLLUTANT_CODES_5474,POLLUTANT_CODES_5617,POLLUTANT_CODES_603,POLLUTANT_CODES_6562,POLLUTANT_CODES_6696,POLLUTANT_CODES_7003,POLLUTANT_CODES_8240,POLLUTANT_CODES_None_x,POLLUTANT_CODES_None_y,POLLUTANT_CODES_None_y
0,0500000026055R0002,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0500000026061R0001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0500000039067R0001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0500018017R5001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0500026025R5001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5,0500026049R5001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
6,0500026091R5001,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
7,0500026099R5001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
8,0500039013R5003,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0900000004013R9356,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [40]:
inspection.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,STATE_EPA_FLAG,COMP_MONITOR_TYPE_CODE,PROGRAM_CODES
0,MI00000000000A1932,2015-06-09 00:00:00,2015,E,PCE,"CAAFESOP, CAASIP"
1,CASJV00006029S2076,2015-06-10 00:00:00,2015,L,FOO,CAATVP
2,CASJV00006039C2587,2015-06-08 00:00:00,2015,L,FOO,CAASIP
3,CASJV00006029S1234,2015-06-10 00:00:00,2015,L,FOO,CAATVP
4,CASJV00006099N7478,2015-06-11 00:00:00,2015,L,FOO,CAATVP


In [27]:
titlev.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,COMP_MONITOR_TYPE_CODE,FACILITY_RPT_DEVIATION_FLAG
0,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
1,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
2,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
3,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
4,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y


In [28]:
stacktest.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,COMP_MONITOR_TYPE_CODE,POLLUTANT_CODES,AIR_STACK_TEST_STATUS_CODE
0,RI0000004400900016,2007-10-03 00:00:00,2007,CST,Volatile organic compounds,NaN
1,CT0000000900900001,2007-01-17 00:00:00,2007,CST,Volatile organic compounds,NaN
2,CT0000000900900001,2007-01-18 00:00:00,2007,CST,Volatile organic compounds,NaN
3,ME0000002301300004,2007-05-30 00:00:00,2007,CST,Volatile organic compounds,NaN
4,MA0000002511900605,2011-09-28 00:00:00,2011,CST,VOLATILE ORGANIC COMPOUNDS (VOCS),PSS


In [29]:
formalact.head()

,PGM_SYS_ID,SETTLEMENT_ENTERED_DATE,SETTLEMENT_ENTERED_DATE_year,ENF_TYPE_CODE,PENALTY_AMOUNT
2,IL000197800AAA,2009-01-27,2009,CIV,0.0
4,IL000021816AAQ,2015-04-07,2015,CIV,0.0
5,OH0000000374010102,2015-04-07,2015,CIV,0.0
6,IL000143065AFM,2015-04-07,2015,CIV,0.0
7,IL000095020ABS,2015-04-07,2015,CIV,0.0


In [30]:
informalact.head()

,PGM_SYS_ID,ACHIEVED_DATE,ACHIEVED_DATE_year,ENF_TYPE_CODE
84,AR0000000506900118,2007-08-10 00:00:00,2007,NOV
86,TX0000004844900005,2008-02-05 00:00:00,2008,NOV
87,LA0000002200500046,2008-06-26 00:00:00,2008,NOV
89,0900000006071R9800,2008-03-18 00:00:00,2008,NOV
90,0900000006099R9800,2008-03-08 00:00:00,2008,NOV
